In [1]:
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader, Dataset
# from PIL import Image
# import os

# # Hyperparameters
# num_epochs = 2
# batch_size = 1
# learning_rate = 0.0002
# image_size = 256

# # Data Transform
# transform = transforms.Compose([
#     transforms.Resize((image_size, image_size)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
# ])

# # Custom Dataset
# class RainDataset(Dataset):
#     def __init__(self, rainy_path, derain_path, transform=None):
#         self.rainy_images = os.listdir(rainy_path)
#         self.derain_images = os.listdir(derain_path)
#         self.rainy_path = rainy_path
#         self.derain_path = derain_path
#         self.transform = transform

#     def __len__(self):
#         return min(len(self.rainy_images), len(self.derain_images))

#     def __getitem__(self, idx):
#         rainy_img = Image.open(os.path.join(self.rainy_path, self.rainy_images[idx]))
#         derain_img = Image.open(os.path.join(self.derain_path, self.derain_images[idx]))
        
#         if self.transform:
#             rainy_img = self.transform(rainy_img)
#             derain_img = self.transform(derain_img)
#         return rainy_img, derain_img

# # Instantiate Dataset and DataLoader
# rainy_path = '/Users/matthew/Jupyter/Thesis/DeRain/JRDR/rain_data_train_Heavy/rain/X2'
# derain_path = '/Users/matthew/Jupyter/Thesis/DeRain/JRDR/rain_data_train_Heavy/norain'
# dataset = RainDataset(rainy_path, derain_path, transform)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Define CycleGAN models (Generator and Discriminator)
# # Generator, Discriminator, CycleGAN Loss functions and Training Loop omitted for simplicity.
# # Refer to a CycleGAN implementation for details on networks and training steps.

# # Training Loop (pseudo-code)
# for epoch in range(num_epochs):
#     for rainy, derain in dataloader:
#         # Forward pass through generators and discriminators
#         # Calculate cycle consistency, adversarial, and identity losses
#         # Backpropagation and weight update
#         pass
#     print(f"Epoch [{epoch+1}/{num_epochs}] completed")

# class Generator(nn.Module):
#     def __init__(self):
#         super(Generator, self).__init__()
#         # Define generator layers (e.g., conv layers, residual blocks)
#         pass

#     def forward(self, x):
#         # Define forward pass
#         return x

# # Initialize generators and discriminators
# generator_rain_to_clear = Generator()
# generator_clear_to_rain = Generator()

# # Save model checkpoints after training
# torch.save(generator_rain_to_clear.state_dict(), "generator_rain_to_clear.pth")
# torch.save(generator_clear_to_rain.state_dict(), "generator_clear_to_rain.pth")


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

# Hyperparameters
num_epochs = 10
batch_size = 1
learning_rate = 0.0002
image_size = 256
lambda_cycle = 10.0
lambda_identity = 5.0

# Data Transform
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Instantiate Dataset and DataLoader
rainy_path = '/Users/matthew/Jupyter/Thesis/DeRain/JRDR/rain_data_test_Heavy/rain/X2'
derain_path = '/Users/matthew/Jupyter/Thesis/DeRain/JRDR/rain_data_test_Heavy/norain'
dataset = RainDataset(rainy_path, derain_path, transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Custom Dataset
class RainDataset(Dataset):
    def __init__(self, rainy_path, derain_path, transform=None):
        self.rainy_images = os.listdir(rainy_path)
        self.derain_images = os.listdir(derain_path)
        self.rainy_path = rainy_path
        self.derain_path = derain_path
        self.transform = transform

    def __len__(self):
        return min(len(self.rainy_images), len(self.derain_images))

    def __getitem__(self, idx):
        rainy_img = Image.open(os.path.join(self.rainy_path, self.rainy_images[idx]))
        derain_img = Image.open(os.path.join(self.derain_path, self.derain_images[idx]))
        
        if self.transform:
            rainy_img = self.transform(rainy_img)
            derain_img = self.transform(derain_img)
        return rainy_img, derain_img

# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels)
        )

    def forward(self, x):
        return x + self.block(x)

# Generator
class Generator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_residual_blocks=9):
        super(Generator, self).__init__()
        model = [
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=1, padding=3),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
        ]
        in_features = 64
        for _ in range(2):
            out_features = in_features * 2
            model += [
                nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(in_features)]
        for _ in range(2):
            out_features = in_features // 2
            model += [
                nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
        model += [nn.Conv2d(64, out_channels, kernel_size=7, stride=1, padding=3), nn.Tanh()]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

# Discriminator (PatchGAN)
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, stride):
            return [
                nn.Conv2d(in_filters, out_filters, kernel_size=4, stride=stride, padding=1),
                nn.InstanceNorm2d(out_filters),
                nn.LeakyReLU(0.2, inplace=True)
            ]

        self.model = nn.Sequential(
            *discriminator_block(in_channels, 64, stride=2),
            *discriminator_block(64, 128, stride=2),
            *discriminator_block(128, 256, stride=2),
            *discriminator_block(256, 512, stride=1),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        )

    def forward(self, img):
        return self.model(img)

# Instantiate models
generator_rain_to_clear = Generator()
generator_clear_to_rain = Generator()
discriminator_rain = Discriminator()
discriminator_clear = Discriminator()

# Losses and optimizers
adversarial_loss = nn.MSELoss()
cycle_loss = nn.L1Loss()
identity_loss = nn.L1Loss()

optimizer_G = optim.Adam(
    list(generator_rain_to_clear.parameters()) + list(generator_clear_to_rain.parameters()), 
    lr=learning_rate, betas=(0.5, 0.999)
)
optimizer_D_rain = optim.Adam(discriminator_rain.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D_clear = optim.Adam(discriminator_clear.parameters(), lr=learning_rate, betas=(0.5, 0.999))

# Training Loop
for epoch in range(num_epochs):
    for i, (rainy, derain) in enumerate(dataloader):
        valid = torch.ones(rainy.size(0), 1, 30, 30, requires_grad=False)
        fake = torch.zeros(rainy.size(0), 1, 30, 30, requires_grad=False)

        # Train Generators
        optimizer_G.zero_grad()
        
        fake_clear = generator_rain_to_clear(rainy)
        loss_GAN_rain_to_clear = adversarial_loss(discriminator_clear(fake_clear), valid)
        
        fake_rain = generator_clear_to_rain(derain)
        loss_GAN_clear_to_rain = adversarial_loss(discriminator_rain(fake_rain), valid)

        recov_rainy = generator_clear_to_rain(fake_clear)
        loss_cycle_rainy = cycle_loss(recov_rainy, rainy)

        recov_derain = generator_rain_to_clear(fake_rain)
        loss_cycle_derain = cycle_loss(recov_derain, derain)
        
        loss_identity_rain = identity_loss(generator_clear_to_rain(rainy), rainy)
        loss_identity_clear = identity_loss(generator_rain_to_clear(derain), derain)

        loss_G = (
            loss_GAN_rain_to_clear + loss_GAN_clear_to_rain + 
            lambda_cycle * (loss_cycle_rainy + loss_cycle_derain) + 
            lambda_identity * (loss_identity_rain + loss_identity_clear)
        )
        loss_G.backward()
        optimizer_G.step()

        # Train Discriminators
        optimizer_D_rain.zero_grad()
        optimizer_D_clear.zero_grad()

        loss_real_rain = adversarial_loss(discriminator_rain(rainy), valid)
        loss_fake_rain = adversarial_loss(discriminator_rain(fake_rain.detach()), fake)
        loss_D_rain = (loss_real_rain + loss_fake_rain) / 2
        loss_D_rain.backward()
        optimizer_D_rain.step()

        loss_real_clear = adversarial_loss(discriminator_clear(derain), valid)
        loss_fake_clear = adversarial_loss(discriminator_clear(fake_clear.detach()), fake)
        loss_D_clear = (loss_real_clear + loss_fake_clear) / 2
        loss_D_clear.backward()
        optimizer_D_clear.step()

        if i % 50 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Batch [{i}/{len(dataloader)}], "
                  f"Loss_G: {loss_G.item():.4f}, Loss_D_rain: {loss_D_rain.item():.4f}, Loss_D_clear: {loss_D_clear.item():.4f}")


# Save models after training
torch.save(generator_rain_to_clear.state_dict(), "/Users/matthew/Jupyter/Thesis/DeRain/generator_rain_to_clear.pth")
torch.save(generator_clear_to_rain.state_dict(), "/Users/matthew/Jupyter/Thesis/DeRain/generator_clear_to_rain.pth")
torch.save(discriminator_rain.state_dict(), "/Users/matthew/Jupyter/Thesis/DeRain/discriminator_rain.pth")
torch.save(discriminator_clear.state_dict(), "/Users/matthew/Jupyter/Thesis/DeRain/discriminator_clear.pth")


Epoch [0/2], Batch [0/200], Loss_G: 25.7280, Loss_D_rain: 0.5443, Loss_D_clear: 0.4726
Epoch [0/2], Batch [50/200], Loss_G: 9.5660, Loss_D_rain: 0.3434, Loss_D_clear: 0.2292
Epoch [0/2], Batch [100/200], Loss_G: 7.2384, Loss_D_rain: 0.2221, Loss_D_clear: 0.3183
Epoch [0/2], Batch [150/200], Loss_G: 12.2940, Loss_D_rain: 0.2300, Loss_D_clear: 0.1803
Epoch [1/2], Batch [0/200], Loss_G: 9.1502, Loss_D_rain: 0.3704, Loss_D_clear: 0.2879
Epoch [1/2], Batch [50/200], Loss_G: 10.9011, Loss_D_rain: 0.1657, Loss_D_clear: 0.2540
Epoch [1/2], Batch [100/200], Loss_G: 6.4483, Loss_D_rain: 0.1857, Loss_D_clear: 0.2956
Epoch [1/2], Batch [150/200], Loss_G: 8.9154, Loss_D_rain: 0.1812, Loss_D_clear: 0.3788
